<center>

# $\textbf{Tax}$

<center>

### $\textbf{Code}$

In [19]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, col, lit, array, struct

In [20]:
spark = SparkSession.builder.appName('Tax').master("local").enableHiveSupport().getOrCreate()
spark

In [21]:
# Creating dataframe from the csv file and infering the schema
df = spark.read.load("Files/Tax2.csv", format="csv", sep=",", inferschema="true", header="true")

In [22]:
# Rename the column
df = df.withColumnRenamed("Country Name","country")
columns_to_drop = [str(year) for year in range(1960, 2000)]
df = df.drop("Country Code","Indicator Name","Indicator Code",*columns_to_drop,"_c67")

In [23]:
# Assuming df is properly defined DataFrame
df = df.select("country", explode(array([
    struct(lit(year).alias("year"), col(str(year)).alias("tax")) 
    for year in range(2000, 2023)
])).alias("data")).selectExpr("country", "data.year", "data.tax")

df = df.orderBy("country")

In [24]:
#Storing this dataframe in parquet
df.write.mode("overwrite").parquet("FilesParquet/Tax.parquet" )
spark.read.parquet("FilesParquet/Tax.parquet").show()
spark.stop()

+-----------+----+----------------+
|    country|year|             tax|
+-----------+----+----------------+
|Afghanistan|2000|            null|
|Afghanistan|2001|            null|
|Afghanistan|2002|            null|
|Afghanistan|2003|            null|
|Afghanistan|2004|            null|
|Afghanistan|2005|            null|
|Afghanistan|2006|6.96759757507017|
|Afghanistan|2007|5.28345660398282|
|Afghanistan|2008|6.08796493020769|
|Afghanistan|2009|8.48175836103604|
|Afghanistan|2010|9.16975191915408|
|Afghanistan|2011|8.91679399595736|
|Afghanistan|2012|7.70780060287773|
|Afghanistan|2013|7.12277329020889|
|Afghanistan|2014|6.88210294277301|
|Afghanistan|2015|7.58538233129798|
|Afghanistan|2016|9.50265278288094|
|Afghanistan|2017|9.89845089688421|
|Afghanistan|2018|            null|
|Afghanistan|2019|            null|
+-----------+----+----------------+
only showing top 20 rows

